# Graph Neural Networks with Deep Graph Library



## Creating a DGL graph from scratch


In [ ]:
!pip install --pre dgl -f https://data.dgl.ai/wheels/cu116/repo.html
!pip install --pre dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/cu116/repo.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels-test/repo.html


In [ ]:
import torch
import dgl

In [ ]:
U = [0, 1, 2, 4, 5, 6, 9]
V = [1, 9, 4, 0, 0, 1, 2]
G = dgl.graph((U, V)) # directed by default
G # nodes 3, 7, 8 are automatically constructed

Graph(num_nodes=10, num_edges=7,
      ndata_schemes={}
      edata_schemes={})

You can create your graphs manually similar to above or you can use external sources (visit the [dgl documentation](https://docs.dgl.ai/guide/graph-external.html))

Graphs functions somewhat like a dictionary. You could add some information to the edges and the nodes.

In [ ]:
G.ndata['features'] = torch.eye(10)
G.ndata['labels'] = torch.randint(3,size=(10,1))
G

Graph(num_nodes=10, num_edges=7,
      ndata_schemes={'features': Scheme(shape=(10,), dtype=torch.float32), 'labels': Scheme(shape=(1,), dtype=torch.int64)}
      edata_schemes={})

To access the features of node 5

In [ ]:
G.ndata['features'][5]

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

Placing graph in your GPU is similar to a placing a tensor.

Note: the node information are also placed in the GPU

In [ ]:
print(G.device)
print(G.ndata['features'].device)
G = G.to('cuda') # G.cuda()
print(G.device)
print(G.ndata['features'].device)

cpu
cpu
cuda:0
cuda:0


## Using Graph Convolutional Networks

We used the Cora node classification dataset. Each node is a scientific publication and edges mean citations. Each publication is classified into one of the 7 subjects: ``Case_Based``,``Genetic_Algorithms``, ``Neural_Networks``, ``Probabilistic_Methods``, ``Reinforcement_Learning``, ``Rule_Learning``, ``Theory``. The node features represent the existence of a specific word in the paper (only 1433 words) with some normalization.

Given this citation network and the node features, we want to classify the subject of the nodes.

In [ ]:
from dgl.data import CoraGraphDataset
dataset = CoraGraphDataset()
G = dataset[0] # this is a Dataset with only one graph

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [ ]:
G

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

In [ ]:
G.ndata['feat']

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
G.ndata['feat'][0, :100]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.1111, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.1111, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000])

In [ ]:
G.ndata['label']

tensor([3, 4, 4,  ..., 3, 3, 3])

In [ ]:
G.ndata['train_mask']

tensor([ True,  True,  True,  ..., False, False, False])

In [ ]:
torch.arange(G.number_of_nodes())[G.ndata['train_mask']]

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139])

In [ ]:
torch.arange(G.number_of_nodes())[G.ndata['val_mask']]

tensor([140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
        182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195,
        196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209,
        210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223,
        224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237,
        238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251,
        252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265,
        266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279,
        280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293,
        294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307,
        308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 3

In [ ]:
print(len(torch.arange(G.number_of_nodes())[G.ndata['train_mask']]))
print(len(torch.arange(G.number_of_nodes())[G.ndata['val_mask']]))
print(len(torch.arange(G.number_of_nodes())[G.ndata['test_mask']]))

140
500
1000


### Defining the Model


In [ ]:
import torch.nn as nn
from dgl.nn import GraphConv # GCN
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_class):
        super(GCN, self).__init__()

        self.conv1 = GraphConv(input_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, num_class)

    def forward(self, g, h):
        h = F.elu(self.conv1(g, h))
        h = self.conv2(g, h)
        return h

### Training the model

In [ ]:
def accuracy(output, labels):
    _, idx = torch.max(output, dim=1)
    correct = torch.sum(idx == labels)
    return correct.item()/len(labels)

def train_whole(G, model, device, epochs):
    g = G.to(device)

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']

    # add self loop since we are using GraphConv
    g = dgl.remove_self_loop(g) # remove self-loops first because we do not want duplicate edges
    g = dgl.add_self_loop(g)
    model = model.to(device)
    ce_loss = torch.nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=5e-4)

    for epoch in range(1, epochs+1):

        model.train()
        optimizer.zero_grad()
        logits = model(g, features) # what is the size of features? number_of_nodes X num_feats --> nodes X 7
        loss = ce_loss(logits[train_mask], labels[train_mask]) # use only training nodes for the loss


        loss.backward()
        optimizer.step()

        train_acc = accuracy(logits[train_mask], labels[train_mask]) # calculate training accuracy

        model.eval()
        with torch.no_grad():
            logits = model(g, features)
            logits = logits[val_mask]
            val_lab = labels[val_mask]
        val_acc = accuracy(logits, val_lab)

        print("Epoch {:03d} | Loss {:.4f} | TrainAcc {:.4f} |"
              " ValAcc {:.4f}".format(epoch, loss.item(), train_acc, val_acc))

    with torch.no_grad():
        logits = model(g, features)
        logits = logits[test_mask]
        test_lab = labels[test_mask]
        acc = accuracy(logits, test_lab)
    print("Test Accuracy {:.4f}".format(acc))

In [ ]:
model = GCN(G.ndata['feat'].shape[1], hidden_dim=8, num_class=7)
train_whole(G, model, 'cuda', 200)

Epoch 001 | Loss 1.9478 | TrainAcc 0.0857 | ValAcc 0.0720
Epoch 002 | Loss 1.9438 | TrainAcc 0.1500 | ValAcc 0.0760
Epoch 003 | Loss 1.9400 | TrainAcc 0.1786 | ValAcc 0.0980
Epoch 004 | Loss 1.9361 | TrainAcc 0.2214 | ValAcc 0.1380
Epoch 005 | Loss 1.9323 | TrainAcc 0.2929 | ValAcc 0.1860
Epoch 006 | Loss 1.9284 | TrainAcc 0.4000 | ValAcc 0.2320
Epoch 007 | Loss 1.9244 | TrainAcc 0.4714 | ValAcc 0.2800
Epoch 008 | Loss 1.9204 | TrainAcc 0.5214 | ValAcc 0.3100
Epoch 009 | Loss 1.9163 | TrainAcc 0.5357 | ValAcc 0.3180
Epoch 010 | Loss 1.9121 | TrainAcc 0.5500 | ValAcc 0.3280
Epoch 011 | Loss 1.9078 | TrainAcc 0.5929 | ValAcc 0.3520
Epoch 012 | Loss 1.9035 | TrainAcc 0.6500 | ValAcc 0.3840
Epoch 013 | Loss 1.8990 | TrainAcc 0.6714 | ValAcc 0.4120
Epoch 014 | Loss 1.8945 | TrainAcc 0.7286 | ValAcc 0.4440
Epoch 015 | Loss 1.8898 | TrainAcc 0.7500 | ValAcc 0.4780
Epoch 016 | Loss 1.8851 | TrainAcc 0.7429 | ValAcc 0.4940
Epoch 017 | Loss 1.8803 | TrainAcc 0.7571 | ValAcc 0.5020
Epoch 018 | Lo